In [ ]:
# List of modules that are not installed in the course
!pip install OpenPermID
!pip install geocoder
!pip install matplotlib
!pip install xlrd==1.2.0
!pip install xlsxwriter
!pip install sklearn
!pip install ipywidgets

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

# Downloading all bonds ever owned in CSPP

In [ ]:
from functions import downloadECBBonds

In [ ]:
holdingsECB = downloadECBBonds.download_ECB_Bonds()

# Read all the Eikon data

In [ ]:
from functions import readEikonData
eikon_data_complete = readEikonData.get_eikon_data_complete()
eligible_complete = readEikonData.get_data_eligible_complete() # read all data from eligible universe

Compare how many bonds from the eligible universe the ECB bought.

In [ ]:
overlap = eikon_data_complete[(eikon_data_complete["ISIN"].isin(eligible_complete["ISIN"]))]
print("the percentage of bonds bought by ECB in eligible universe that we have info on:    ", 
      overlap.shape[0]/eligible_complete.shape[0])

Read the dates

In [ ]:
years_issuer_bought = readEikonData.get_dates_data_frame()

# ECB green bonds that are listed on Euronext stock exchange 

Comparing euronext completeness of data to EIKON data. 

In [ ]:
from functions import greenbonds
greenbonds.compareGreenbondsEuronextvsEIKON()

EIKON data is more complete than euronext

Plot of percentage of greenbonds in comparison to all bonds per year

In [ ]:
greenbonds.draw_spaghetti_plot_greenbonds()

# Company Analysis based on ESG scores

Function for cleaning the ESG data

In [ ]:
from functions import ESG
print("for eligible universe:")
esg_company_data_eligible = ESG.cleaning_esg_data(eligible_complete)
print("for holdings in the CSPP:")
esg_company_data_holdings = ESG.cleaning_esg_data(eikon_data_complete)

## ESG Evolution of holdings through the years

In [ ]:
ESG.average_company_esg_score_plot(years_issuer_bought, esg_company_data_holdings,esg_company_data_eligible)

# Sector Analysis

## Get all sector data from PermID

In [ ]:
from functions import permidSectors
sector_mappings = permidSectors.get_sector_mappings(holdingsECB)
sector_mappings.dropna(inplace = True)

## Represent industries and sectors

Create a pie chart to represent the number of times an industry, sector or supersector is present in the data (multiple bonds for the same company are counted separately)

In [ ]:
from functions import pieCharts

In [ ]:
pieCharts.make_pie_chart('hasPrimaryBusinessSector', sector_mappings)

In [ ]:
pieCharts.make_pie_chart('hasPrimaryEconomicSector', sector_mappings)

In [ ]:
pieCharts.make_pie_chart('hasPrimaryIndustryGroup', sector_mappings)

## Which industries and sectors are green?

Map the dictionary with greenness of all PermID sectors with the ones in which ECB invested



In [ ]:
from functions import sectors
sector_green_dict = sectors.get_sector_green_dict()

In [ ]:
primary_business_sector = pieCharts.get_all_sectors("hasPrimaryBusinessSector", sector_mappings)
primary_business_sector = sectors.map_green_dict_to_data_frame(primary_business_sector)

- Total Count for each sector overall and compare these sectors (we view as “bad”, like cement, oil&gas or with kaggle dataset(emission))
- Similar approach but per year → do we see a change in habit, total counts → differences in the sectors 
- For the sectors: make spaghetti plot with for each sector the percentage invested in every year (in app: select which sectors you want)
    - Include in app (can view all, and have “buttons” to choose what to show) 
    - Make percentages instead of absolute counts
    - Remove before 2017

In [ ]:
sector_mappings["Greenness"] = sector_mappings["hasPrimaryBusinessSector"].dropna().apply(lambda x: sector_green_dict[x])

In [ ]:
# Draw a spaghetti plot for all the sectors
sectors.draw_spaghetti_plot_sectors(primary_business_sector, sector_mappings, years_issuer_bought)

In [ ]:
# Draw a spaghetti plot for the grey sectors
grey_sectors = primary_business_sector[primary_business_sector["green"]==-1].index
sectors.draw_spaghetti_plot_sectors(primary_business_sector, sector_mappings, years_issuer_bought, grey_sectors)

In [ ]:
# Draw a spaghetti plot for the green sectors
green_sectors = primary_business_sector[primary_business_sector["green"]==1].index
sectors.draw_spaghetti_plot_sectors(primary_business_sector, sector_mappings, years_issuer_bought, green_sectors)

In [ ]:
# Drawing a sphagetti plot by grouping sectors according to greenness
green_dict = {-1: "Less green", 0: "Neutral", 1: "More green"}
sector_mappings_modified = sector_mappings.copy()
sector_mappings_modified["hasPrimaryBusinessSector"] = sector_mappings_modified.Greenness.apply(lambda x: green_dict[x])
levels_of_greenness = pieCharts.get_all_sectors("hasPrimaryBusinessSector", sector_mappings_modified)
levels_of_greenness = sectors.map_green_dict_to_data_frame(levels_of_greenness)
sectors.draw_spaghetti_plot_sectors(levels_of_greenness, sector_mappings_modified, years_issuer_bought, title = "greenness groups")

In [ ]:
from IPython.display import (display, clear_output)
import ipywidgets as widgets

quick_pick = widgets.RadioButtons(
    options=["All", "Green", "Grey"],
    description="Sectors: "
)

customize = widgets.SelectMultiple(
    options= list(sectors.get_sector_green_dict().keys()),
    value=["Energy - Fossil Fuels"],
    rows=33,
    description="Sectors: ",
    layout={'width': 'max-content'},
    disabled=False
)
button_all = widgets.Button(description = 'Select all')

def select_all(*args):
    customize.value = customize.options
    quick_pick.value = "All"

tab_widget = widgets.Tab()
tab_widget.children = [quick_pick, customize]
tab_widget.set_title(0, "Quick pick")
tab_widget.set_title(1, "Customize")

output = widgets.Output()

display(tab_widget, button_all, output)

def getSectorNames(category):
    switcher={
            "Green": green_sectors,
            "Grey": grey_sectors
            }
    return switcher.get(category, pd.DataFrame())

def on_value_change(change):
    with output:
        # Draw a spaghetti plot for all the sectors
        clear_output()
        sectors.draw_spaghetti_plot_sectors(primary_business_sector, sector_mappings, years_issuer_bought, getSectorNames(change["new"]))

button_all.on_click(select_all)
quick_pick.observe(on_value_change, names = "value")